# Word2Vec

In [15]:
import pandas as pd

# Load data (LIAR Dataset)
data_df_train_raw = pd.read_csv("../../dataset/LIAR/data/train.csv", sep=",", header=0)
data_df_test_raw = pd.read_csv("../../dataset/LIAR/data/test.csv", sep=",", header=0)
data_df_valid_raw = pd.read_csv("../../dataset/LIAR/data/valid.csv", sep=",", header=0)


In [16]:
# Merge all data
master_df = pd.concat(
    [data_df_train_raw, data_df_test_raw, data_df_valid_raw], ignore_index=True
)
print(master_df.shape)
master_df.head(5)


(12791, 15)


,id,label,statement,subject,speaker,job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,statement_processed
0,2635.json,False,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,say Annies List political group support trimes...
1,10540.json,True,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,decline coal start start natural gas take star...
2,324.json,True,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Hillary Clinton agree John McCain vote George ...
3,1123.json,False,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,health care reform legislation likely mandate ...
4,9028.json,True,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,economic turnaround start end term


## Download Pre-trained model  

Ref: https://code.google.com/archive/p/word2vec/  

Download Link: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g

In [17]:
import gensim

pretrained_model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

In [18]:
import numpy as np

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

# Ticket Data
corpus = master_df['statement_processed'].values

# Creating the vectorizer
vectorizer = CountVectorizer()

# Converting the text to numeric data
X = vectorizer.fit_transform(corpus)

#print(vectorizer.get_feature_names())

# Preparing Data frame For machine learning
# Priority column acts as a target variable and other columns as predictors
vector_df=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
print(vector_df.shape)
vector_df.head(5)

(12791, 10491)


,00,000,000new,014,02,024,029,033,036,04,...,zip,zippo,zombie,zone,zones,zoning,zoo,zuckerberg,zuckerbergs,ʺmore
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
coprus_vector = vector_df.columns


def get_similarity_from_pretrained_model(inpTextData):
    vector = vectorizer.transform(inpTextData)
    vec_count = pd.DataFrame(vector.toarray(), columns=vectorizer.get_feature_names_out())

    w2v = pd.DataFrame()

    for i in range(vec_count.shape[0]):
        # Zero Padding
        Sentence = np.zeros(300) 

        for word in coprus_vector[vec_count.iloc[i, :] >= 1]:
            if word in pretrained_model.key_to_index.keys():
                Sentence = Sentence + pretrained_model[word]
        w2v = w2v.append(pd.DataFrame([Sentence]))
    return w2v


In [21]:
w2v_X = get_similarity_from_pretrained_model(master_df['statement_processed'])

/tmp/ipykernel_5402/1823804509.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  w2v = w2v.append(pd.DataFrame([Sentence]))
/tmp/ipykernel_5402/1823804509.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  w2v = w2v.append(pd.DataFrame([Sentence]))
/tmp/ipykernel_5402/1823804509.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  w2v = w2v.append(pd.DataFrame([Sentence]))
/tmp/ipykernel_5402/1823804509.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  w2v = w2v.append(pd.DataFrame([Sentence]))
/tmp/ipykernel_5402/1823804509.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

In [22]:
X=w2v_X
y=master_df['label']

In [23]:
# Split the data into training and testing set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Sanity check for the sampled data
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(10232, 300) (2559, 300) (10232,) (2559,)


## Machine Learning Approach

In [29]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier()
result_rf = RandomForestClassifier().fit(X_train, y_train)

In [30]:
from sklearn.metrics import classification_report

print(classification_report(y_test, result_rf.predict(X_test)))

              precision    recall  f1-score   support

       False       0.59      0.37      0.45      1178
        True       0.59      0.78      0.67      1381

    accuracy                           0.59      2559
   macro avg       0.59      0.57      0.56      2559
weighted avg       0.59      0.59      0.57      2559



In [31]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB

clf_nb = BernoulliNB()
result_nb = clf_nb.fit(X_train, y_train)
print("BernoulliNB")
print(classification_report(y_test, result_nb.predict(X_test)))

clf_gnb = GaussianNB()
result_gnb = clf_gnb.fit(X_train, y_train)
print("GaussianNB")
print(classification_report(y_test, result_gnb.predict(X_test)))


BernoulliNB
              precision    recall  f1-score   support

       False       0.48      0.01      0.02      1178
        True       0.54      0.99      0.70      1381

    accuracy                           0.54      2559
   macro avg       0.51      0.50      0.36      2559
weighted avg       0.51      0.54      0.39      2559

GaussianNB
              precision    recall  f1-score   support

       False       0.49      0.71      0.58      1178
        True       0.60      0.37      0.45      1381

    accuracy                           0.53      2559
   macro avg       0.54      0.54      0.52      2559
weighted avg       0.55      0.53      0.51      2559



In [32]:
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000)
result_lr = clf_lr.fit(X_train, y_train)

print("LogisticRegression")
print(classification_report(y_test, result_lr.predict(X_test)))

LogisticRegression
              precision    recall  f1-score   support

       False       0.60      0.42      0.49      1178
        True       0.61      0.77      0.68      1381

    accuracy                           0.61      2559
   macro avg       0.61      0.59      0.59      2559
weighted avg       0.61      0.61      0.59      2559

